In [1]:
from konfuzio_sdk.data import Project, Annotation, Span, Data, Document, Page
import os
from konfuzio_sdk.api import delete_document_annotation
import json
from typing import Dict, Union, Optional, List
from keras.models import load_model
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from konfuzio_sdk.trainer import file_splitting

In [4]:
file_splitter = file_splitting.PageSplitting('testing_class/testing_class.pickle')

2022-10-18 18:07:12.755800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
file_splitter = file_splitting.FusionPageSplittingModel(project_id=46)

In [5]:
train_data_generator = file_splitter._preprocess_data_vgg16(file_splitter.train_docs)

Found 13 images belonging to 2 classes.


In [6]:
vgg16 = file_splitter._init_vgg16()

In [7]:
vgg16 = file_splitter._train_vgg16(vgg16, train_data_generator)

2022-10-18 18:13:52,773 [tensorflow          ] [MainThread] [WARNING ] [__init__            ][1322] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
/Users/macbookpro/konfuzio-sdk/konfuzio_sdk/trainer/file_splitting.py:117: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(steps_per_epoch=100, generator=train_data_generator,


Epoch 1/100
  1/100 [..............................] - ETA: 32:28 - loss: 0.6328 - accuracy: 1.0000

2022-10-18 18:14:13,820 [tensorflow          ] [MainThread] [WARNING ] [catch_stop_iteration][1232] Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 10000 batches). You may need to use the repeat() function when building your dataset.
2022-10-18 18:14:13,822 [tensorflow          ] [MainThread] [WARNING ] [_save_model         ][1430] Can save best model only with val_accuracy available, skipping.
2022-10-18 18:14:13,823 [tensorflow          ] [MainThread] [WARNING ] [get_monitor_value   ][1887] Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy


100/100 [==============================] - 21s 11ms/step - loss: 0.6328 - accuracy: 1.0000


In [8]:
bert_model, bert_tokenizer = file_splitter._init_legalbert()

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [10]:
Xtrain, Xtest, ytrain, ytest, input_shape = file_splitter._preprocess_data_fusion_model(file_splitter.train_docs, file_splitter.test_docs,
                                                                                       vgg16, bert_model, bert_tokenizer)

2022-10-18 23:23:39,433 [tensorflow          ] [MainThread] [WARNING ] [_conform_to_referenc][0672] Model was constructed with shape (None, 224, 224, 3) for input KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='conv2d_input'), name='conv2d_input', description="created by layer 'conv2d_input'"), but it was called on an input with incompatible shape (None, 224, 224).


ValueError: in user code:

    File "/opt/anaconda3/envs/konfuzio/lib/python3.8/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/konfuzio/lib/python3.8/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/konfuzio/lib/python3.8/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/anaconda3/envs/konfuzio/lib/python3.8/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/opt/anaconda3/envs/konfuzio/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/envs/konfuzio/lib/python3.8/site-packages/keras/engine/input_spec.py", li